In [ ]:
import os
import win32com.client
import numpy as np
import re

def replace_D(s, new_number):
    # Reverse the string to process from the back
    reversed_s = s[::-1]
    # Find the positions of the operators in the reversed string
    operator_positions = [m.start() for m in re.finditer(r'[\+\-]', reversed_s)]
    # The position of the second operator from the end in the original string
    second_operator_pos = operator_positions[1]
    # Reverse the position to get the correct index in the original string
    index_in_original = len(s) - second_operator_pos - 1
    # Find the end position of the number following the second operator
    next_operator_match = re.search(r'[\+\-]', s[index_in_original+1:])
    if next_operator_match:
        end_pos = index_in_original + next_operator_match.start() + 1
    else:
        end_pos = len(s)
    # Replace the second-to-last element including its sign
    new_string = s[:index_in_original] + str(new_number) + s[end_pos:]
    return new_string

#define groups through which to access all utility functions
VHG_EinP = ['EinpA_bF','EinpA_kF','EinpA_nC','EinpA_nE','EinpF_bF','EinpF_kF','EinpF_nC','EinpF_nE','EinpK_bF','EinpK_kF','EinpK_nC','EinpK_nE','EinpU_bF','EinpU_kF','EinpU_nC','EinpU_nE',]
Mode = ['cbike','ebike']

VHG_main = [
    "BSch_bF", "BSch_kF", "BSch_nC", "BSch_nE",
    "ERWmP_bF", "ERWmP_kF", "ERWmP_nC", "ERWmP_nE",
    "ERWoP_bF", "ERWoP_kF", "ERWoP_nC", "ERWoP_nE",
    "GSch_bF", "GSch_kF", "GSch_nC", "GSch_nE",
    "Kind_bF", "Kind_kF", "Kind_nC", "Kind_nE",
    "NERWmP_bF", "NERWmP_kF", "NERWmP_nC", "NERWmP_nE",
    "NERWoP_bF", "NERWoP_kF", "NERWoP_nC", "NERWoP_nE",
    "Rentner_bF", "Rentner_kF", "Rentner_nC", "Rentner_nE",
    "Sch_bF", "Sch_kF", "Sch_nC", "Sch_nE",
    "Stud_bF", "Stud_kF", "Stud_nC", "Stud_nE"
]
Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]

#Parameter values for D to try out (see Excel 003)
D = [['+1.9'],
     ['+2.6'],
     ['+0.9'],
     ['+0.9'],
     ['+1.1'],
     ['+1.9'],
     ['+2.2'],
     ['+1'],
     ['+3.9'],
     ['+3.7'],
     ['-0.9'],
     ['-1.6'],
     ['-1.9'],
     ['-15'],
     ['-25'],
     ['-2'],
     ['-1'],
     ['-3.1'],
     ['-35'],
     ['+0.1']]
# D[3][0]: 4th row (cbike NERWoP), 1st column (0th iteration)

# Matrix numbers for calculating resulting modal share per person group
mBSch_cbike = 5247
mBSch_ebike = 5248
mBSch_other = [5245, 5246, 5249, 5250]

mERWmP_cbike = 5253
mERWmP_ebike = 5254
mERWmP_other = [5251, 5252, 5255, 5256]

mERWoP_cbike = 5259
mERWoP_ebike = 5260
mERWoP_other = [5257, 5258, 5261, 5262]

mNERWmP_cbike = 5277
mNERWmP_ebike = 5278
mNERWmP_other = [5275, 5276, 5279, 5280]

mNERWoP_cbike = 5283
mNERWoP_ebike = 5284
mNERWoP_other = [5281, 5282, 5285, 5286]

mBSch_cbike = 5247
mBSch_ebike = 5248
mBSch_other = [5245, 5246, 5249, 5250]

mGSchSch_cbike = [5265, 5295]
mGSchSch_ebike = [5266, 5296]
mGSchSch_other = [5263, 5264, 5267, 5268, 5293, 5294, 5297, 5298]

mStud_cbike = 5301
mStud_ebike = 5302
mStud_other = [5299, 5300, 5303, 5304]

mRentner_cbike = 5289
mRentner_ebike = 5290
mRentner_other = [5287, 5288, 5291, 5292]

mKind_cbike = 5271
mKind_ebike = 5272
mKind_other = [5269, 5270, 5273, 5274]

mEinp_cbike = 42
mEinp_ebike = 43
mEinp_other = [39, 40, 41]

In [ ]:
#Loop over iteration steps to change D, rerun model, compute mode shares by distance classes, and save
for i in range(0,1):
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250310_0830_D20_0.ver'
    #verPath = r'D:\ver\20240709_1200.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    
    #change utils for Einpendler visem
    visem_einp = Visum.Procedures.Operations.ItemByKey(46)
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":
                if mode == "cbike":
                    new_D = D[9][i]
                elif mode == "ebike":
                    new_D = D[19][i]

                # Set D for all Einp utility functions to value from D (not calibrated, only set similar to value of other groups
                new_util = replace_D(util,new_D)
                #write into Visum
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)
                
    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(47)
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exception
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    if util != "-99999":
                        if mode == "cbike":
                            if vhg =="ERWmP_bF" or vhg =="ERWmP_kF" or vhg =="ERWmP_nC" or vhg =="ERWmP_nE":
                                new_D = D[0][i]
                            elif vhg =="ERWoP_bF" or vhg =="ERWoP_kF" or vhg =="ERWoP_nC" or vhg =="ERWoP_nE":
                                new_D = D[1][i]
                            elif vhg =="NERWmP_bF" or vhg =="NERWmP_kF" or vhg =="NERWmP_nC" or vhg =="NERWmP_nE":
                                new_D = D[2][i]
                            elif vhg =="NERWoP_bF" or vhg =="NERWoP_kF" or vhg =="NERWoP_nC" or vhg =="NERWoP_nE":
                                new_D = D[3][i]
                            elif vhg =="BSch_bF" or vhg =="BSch_kF" or vhg =="BSch_nC" or vhg =="BSch_nE":
                                new_D = D[4][i]
                            elif vhg =="GSch_bF" or vhg =="Sch_bF" or vhg =="GSch_kF" or vhg =="Sch_kF" or vhg =="GSch_nC" or vhg =="Sch_nC" or vhg =="GSch_nE" or vhg =="Sch_nE":
                                new_D = D[5][i]
                            elif vhg =="Stud_bF" or vhg =="Stud_kF" or vhg =="Stud_nC" or vhg =="Stud_nE":
                                new_D = D[6][i]
                            elif vhg =="Rentner_bF" or vhg =="Rentner_kF" or vhg =="Rentner_nC" or vhg =="Rentner_nE":
                                new_D = D[7][i]
                            elif vhg =="Kind_bF" or vhg =="Kind_kF" or vhg =="Kind_nC" or vhg =="Kind_nE":
                                new_D = D[8][i]
                        elif mode == "ebike":
                            if vhg =="ERWmP_bF" or vhg =="ERWmP_kF" or vhg =="ERWmP_nC" or vhg =="ERWmP_nE":
                                new_D = D[10][i]
                            elif vhg =="ERWoP_bF" or vhg =="ERWoP_kF" or vhg =="ERWoP_nC" or vhg =="ERWoP_nE":
                                new_D = D[11][i]
                            elif vhg =="NERWmP_bF" or vhg =="NERWmP_kF" or vhg =="NERWmP_nC" or vhg =="NERWmP_nE":
                                new_D = D[12][i]
                            elif vhg =="NERWoP_bF" or vhg =="NERWoP_kF" or vhg =="NERWoP_nC" or vhg =="NERWoP_nE":
                                new_D = D[13][i]
                            elif vhg =="BSch_bF" or vhg =="BSch_kF" or vhg =="BSch_nC" or vhg =="BSch_nE":
                                new_D = D[14][i]
                            elif vhg =="GSch_bF" or vhg =="Sch_bF" or vhg =="GSch_kF" or vhg =="Sch_kF" or vhg =="GSch_nC" or vhg =="Sch_nC" or vhg =="GSch_nE" or vhg =="Sch_nE":
                                new_D = D[15][i]
                            elif vhg =="Stud_bF" or vhg =="Stud_kF" or vhg =="Stud_nC" or vhg =="Stud_nE":
                                new_D = D[16][i]
                            elif vhg =="Rentner_bF" or vhg =="Rentner_kF" or vhg =="Rentner_nC" or vhg =="Rentner_nE":
                                new_D = D[17][i]
                            elif vhg =="Kind_bF" or vhg =="Kind_kF" or vhg =="Kind_nC" or vhg =="Kind_nE":
                                new_D = D[18][i]
                                
                        #write new utilityfunction
                        new_util = replace_D(util,new_D)
                        #write into Visum
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)
                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    #run procedures
    #Visum.Procedures.Execute() 
    
    # Calculate mode share per persongroup
    print('THIS IS FOR ITERATION NUMBER '+str(i))
    print('mERWmP_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWmP_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWmP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mERWmP_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWmP_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWmP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mERWoP_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWoP_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWoP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mERWoP_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWoP_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWoP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWmP_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWmP_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWoP_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWoP_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mBSch_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mBSch_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mBSch_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mBSch_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mBSch_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mBSch_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mGSchSch_cbike')
    print(round((np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[1]).GetValues()).sum())/
          (np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[3]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[4]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[5]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[6]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[7]).GetValues()).sum())*100, 1))
    print('')

    print('mGSchSch_ebike')
    print(round((np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[1]).GetValues()).sum())/
          (np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_cbike[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_ebike[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[3]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[4]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[5]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[6]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[7]).GetValues()).sum())*100, 1))
    print('')

    print('mStud_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mStud_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mStud_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[3]).GetValues()).sum())*100, 1))
    print('')
    print('mStud_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mStud_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mStud_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mRentner_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mRentner_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mRentner_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mRentner_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mRentner_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mRentner_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mKind_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mKind_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mKind_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mKind_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mKind_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mKind_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mEinp_cbike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mEinp_cbike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mEinp_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[2]).GetValues()).sum())*100, 1))
    print('')

    print('mEinp_ebike')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mEinp_ebike).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mEinp_cbike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_ebike).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[2]).GetValues()).sum())*100, 1))
    print('')
    
    #save
    #verPath_append = r'D:\ver\20240709_1200.ver'#+str(i)+'.ver'
    verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250310_0830_D20_'+str(i)+'.ver'
    #Visum.saveVersion(verPath_append)
    Visum = None